In [3]:
import zeep
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

/Users/sundipta/venv_insight/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Creating DataFrame 1 of yay and nay votes:
        BillID    Vote Value     Legislator

In [22]:
leg_serv = 'http://wslwebservices.leg.wa.gov/LegislationService.asmx?WSDL'
client = zeep.Client(wsdl=leg_serv)

In [23]:
bills_2017 = client.service.GetLegislationByYear('2017')
bills_2018 = client.service.GetLegislationByYear('2018')

In [24]:
bills_curr_bien = bills_2017 + bills_2018

In [25]:
len(bills_curr_bien)

8378

In [26]:
#this cell is super slow. don't run it a lot
# Try using multiprocessing Queue
house_records_currbien = []
senate_records_currbien = []
i = 0
for bill in bills_curr_bien: 
    bill_number = bill['BillNumber']
    roll_call = client.service.GetRollCalls('2017-18', bill_number)
    if roll_call is None:
        i += 1
    else:
        people_dict = {roll['Name']: roll['VOte'] for roll in roll_call[0]['Votes']['Vote']}    
        big_dict = {bill_number : people_dict}
        if roll_call[0]['Agency'] =='Senate':
            senate_records_currbien.append(big_dict)
        elif roll_call[0]['Agency'] =='House':
            house_records_currbien.append(big_dict)
        else:
            print(roll_call[0]['Agency'])
print(i)
%time

5579
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [27]:
senate_billids = []
senate_rows = []
for rec in senate_records_currbien: 
    iden, row = list(rec.items())[0]
    senate_billids.append(iden)
    senate_rows.append(row)

In [28]:
senate_bien_df = pd.DataFrame(senate_rows, index = senate_billids)
senate_bien_bool_df = senate_bien_df.replace({'Yea':1.0, "Nay":0.0,"Excused":np.nan,"Absent":np.nan})

In [29]:
trans_senate = senate_bien_bool_df.transpose()
trans_senate.head()

,1001,1010,1010,1017,1017,1018,1022,1022,1027,1027,...,9301,9302,9303,9321,9333,9336,9338,9339,9340,9800
Angel,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Bailey,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Baumgartner,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN
Becker,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Billig,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [30]:
senate_dict_list = []
for leg, row in trans_senate.iterrows(): 
    for bill, vote in row.iteritems():
        senate_dict = {'Bill_ID': bill,
                      'Legislator':leg,
                      'Vote': vote,
                       'Agency': 'Senate'}
        senate_dict_list.append(senate_dict)

In [31]:
senate_votes_df = pd.DataFrame(senate_dict_list)
senate_votes_df.head()

,Agency,Bill_ID,Legislator,Vote
0,Senate,1001,Angel,1.0
1,Senate,1010,Angel,1.0
2,Senate,1010,Angel,1.0
3,Senate,1017,Angel,1.0
4,Senate,1017,Angel,1.0


In [32]:
house_billids = []
house_rows = []
for rec in house_records_currbien: 
    iden, row = list(rec.items())[0]
    house_billids.append(iden)
    house_rows.append(row)

In [33]:
house_bien_df = pd.DataFrame(house_rows, index = house_billids)
house_bien_bool_df = house_bien_df.replace({'Yea':1.0, "Nay":0.0,"Excused":np.nan,"Absent":np.nan})

In [34]:
house_bien_bool_df.head()

,Appleton,Barkis,Bergquist,Blake,Buys,Caldier,Chandler,Chapman,Chopp,Clibborn,...,Taylor,Tharinger,Valdez,Van Werven,Vick,Volz,Walsh,Wilcox,Wylie,Young
1031,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
1032,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0
1037,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1037,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1045,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
trans_house = house_bien_bool_df.transpose()
trans_house.head()

,1031,1032,1037,1037,1045,1045,1046,1046,1053,1054,...,2928,2961,2962,2975,2975,4008,4010,4011,4012,4014
Appleton,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Barkis,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Bergquist,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Blake,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Buys,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [36]:
house_dict_list = []
for leg, row in trans_house.iterrows(): 
    for bill, vote in row.iteritems():
        house_dict = {'Bill_ID': bill,
                      'Legislator':leg,
                      'Vote': vote,
                       'Agency': 'House'}
        house_dict_list.append(house_dict)

In [37]:
house_votes_df = pd.DataFrame(house_dict_list)
house_votes_df.head()

,Agency,Bill_ID,Legislator,Vote
0,House,1031,Appleton,1.0
1,House,1032,Appleton,1.0
2,House,1037,Appleton,1.0
3,House,1037,Appleton,1.0
4,House,1045,Appleton,1.0


In [38]:
curr_bien_votes_df = house_votes_df.append(senate_votes_df)
curr_bien_votes_df.head()

,Agency,Bill_ID,Legislator,Vote
0,House,1031,Appleton,1.0
1,House,1032,Appleton,1.0
2,House,1037,Appleton,1.0
3,House,1037,Appleton,1.0
4,House,1045,Appleton,1.0


In [39]:
curr_bien_votes_df_clean = curr_bien_votes_df.drop_duplicates(subset=None, keep='first', inplace=False)

In [40]:
curr_bien_votes_df_clean.to_csv('current_bien_votes.csv', sep = ',')

In [41]:
#dictionary of lists is columns
#list of dictionarys is rows

### Making DataFrame 2 of bill descriptions:
    BillID    short desc     long description  Sponsor

In [42]:
house_billinfo_bien = []
senate_billinfo_bien = []
i = 0
for bill in bills_curr_bien: 
    bill_number = bill['BillNumber']
    roll_call = client.service.GetRollCalls('2017-18', bill_number)
    if roll_call is None:
        i += 1
    else:
        bill_info = client.service.GetLegislation('2017-18', bill_number)
        short_desc = bill_info[0]['ShortDescription']
        long_desc = bill_info[0]['LongDescription']
        bill_sponsor = bill_info[0]['Sponsor']
        activeness = bill['Active']
        if roll_call[0]['Agency'] =='Senate':
            sen_dict = {'Bill_ID': bill_number,
                      'ShortDescription':short_desc,
                      'LongDescription': long_desc,
                       'Agency': 'Senate',
                       'Active': activeness}
            senate_billinfo_bien.append(sen_dict)
        elif roll_call[0]['Agency'] =='House':
            house_dict = {'Bill_ID': bill_number,
                      'ShortDescription':short_desc,
                      'LongDescription': long_desc,
                       'Agency': 'House',
                       'Active': activeness}
            house_billinfo_bien.append(house_dict)
        else:
            print(roll_call[0]['Agency'])
    
print(i)

5579


In [43]:
#set up a foreign key for these tables in sql

In [44]:
house_bill_info_df =  pd.DataFrame(house_billinfo_bien)
senate_bill_info_df =  pd.DataFrame(senate_billinfo_bien)

In [45]:
bill_info_df = house_bill_info_df.append(senate_bill_info_df)
bill_info_df

,Active,Agency,Bill_ID,LongDescription,ShortDescription
0,True,House,1031,Concerning the use of unmanned aerial systems ...,UAVs near marine species
1,True,House,1032,Concerning the excise taxation of martial arts.,Martial arts excise taxation
2,False,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
3,True,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
4,False,House,1045,Addressing licensing and enforcement provision...,Uniform money services act
5,True,House,1045,Addressing licensing and enforcement provision...,Uniform money services act
6,False,House,1046,Concerning certificates of academic and indivi...,Certificates of achievement
7,True,House,1046,Concerning certificates of academic and indivi...,Certificates of achievement
8,True,House,1053,Addressing the Washington state credit union act.,WA state credit union act
9,False,House,1054,Concerning the age of individuals at which sal...,Tobacco & vapor products/age


In [46]:
bill_info_df_clean = bill_info_df.drop_duplicates(subset=None, keep='first', inplace=False)

In [47]:
bill_info_df_clean.to_csv('bill_descriptions.csv', sep = ',')

### Making DataFrame 3 of Legislator information

In [49]:
house_people_info = pd.read_csv("/Users/sundipta/Insight_notebooks/house_rosters-2018.csv")

In [50]:
house_people_info.head()

,Member Name,Party,District,Position,Room,Phone,Email,Chamber,Legislative Assistant,LA Email
0,Sherry Appleton,D,23,1,LEG 132F,(360) 786-7934,Sherry.Appleton@leg.wa.gov,House,Lisa Hardy,Lisa.Hardy@leg.wa.gov
1,Andrew Barkis,R,2,1,LEG 122D,(360) 786-7824,Andrew.Barkis@leg.wa.gov,House,Shelby Pelon,Shelby.Pelon@leg.wa.gov
2,Steve Bergquist,D,11,2,JLOB 322,(360) 786-7862,Steve.Bergquist@leg.wa.gov,House,Syd Locke,Syd.Locke@leg.wa.gov
3,Brian Blake,D,19,2,LEG 437A,(360) 786-7870,Brian.Blake@leg.wa.gov,House,Dawn Thomas,Dawn.Thomas@leg.wa.gov
4,Vincent Buys,R,42,2,JLOB 465,(360) 786-7854,Vincent.Buys@leg.wa.gov,House,Amanda Yager,Amanda.Yager@leg.wa.gov


In [52]:
senate_people_info = pd.read_csv("/Users/sundipta/Insight_notebooks/senate_rosters-2018.csv")

In [53]:
senate_people_info

,Member Name,Party,District,Position,Room,Phone,Email,Chamber,Legislative Assistant,LA Email,Unnamed: 10,Unnamed: 11
0,Jan Angel,R,26,NaN,INB 203,(360) 786-7650,Jan.Angel@leg.wa.gov,Senate,Holly Cocci,Holly.Cocci@leg.wa.gov,NaN,NaN
1,Barbara Bailey,R,10,NaN,LEG 407,(360) 786-7618,Barbara.Bailey@leg.wa.gov,Senate,Vicki Angelini,Vicki.Angelini@leg.wa.gov,NaN,NaN
2,Michael Baumgartner,R,6,NaN,LEG 404,(360) 786-7610,Michael.Baumgartner@leg.wa.gov,Senate,Matt McCoy,Matt.McCoy@leg.wa.gov,NaN,NaN
3,Randi Becker,R,2,NaN,LEG 316,(360) 786-7602,Randi.Becker@leg.wa.gov,Senate,Tiffani Sann̩,Tiffani.Sanne@leg.wa.gov,Eden Craig-Mickel,Eden.Craig-Mickel@leg.wa.gov
4,Andy Billig,D,3,NaN,LEG 412,(360) 786-7604,Andy.Billig@leg.wa.gov,Senate,Noelle Connolly,Noelle.Connolly@leg.wa.gov,NaN,NaN
5,John Braun,R,20,NaN,LEG 403A,(360) 786-7638,John.Braun@leg.wa.gov,Senate,Jeri May,Jeri.May@leg.wa.gov,Ruth Peterson,Ruth.Peterson@leg.wa.gov
6,Sharon Brown,R,8,NaN,INB 202,(360) 786-7614,Sharon.Brown@leg.wa.gov,Senate,Reni Michael,Reni.Michael@leg.wa.gov,NaN,NaN
7,Reuven Carlyle,D,36,NaN,JAC 233,(360) 786-7670,Reuven.Carlyle@leg.wa.gov,Senate,Kate Hoffman,Kate.Hoffman@leg.wa.gov,NaN,NaN
8,Maralyn Chase,D,32,NaN,JAC 224,(360) 786-7662,Maralyn.Chase@leg.wa.gov,Senate,Remy Golla,Remy.Golla@leg.wa.gov,NaN,NaN
9,Annette Cleveland,D,49,NaN,JAC 220,(360) 786-7696,Annette.Cleveland@leg.wa.gov,Senate,Vickie Winters,Vickie.Winters@leg.wa.gov,NaN,NaN


In [54]:
legislator_info = house_people_info.append(senate_people_info)

/Users/sundipta/venv_insight/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [55]:
legislator_info

,Chamber,District,Email,LA Email,Legislative Assistant,Member Name,Party,Phone,Position,Room,Unnamed: 10,Unnamed: 11
0,House,23,Sherry.Appleton@leg.wa.gov,Lisa.Hardy@leg.wa.gov,Lisa Hardy,Sherry Appleton,D,(360) 786-7934,1.0,LEG 132F,NaN,NaN
1,House,2,Andrew.Barkis@leg.wa.gov,Shelby.Pelon@leg.wa.gov,Shelby Pelon,Andrew Barkis,R,(360) 786-7824,1.0,LEG 122D,NaN,NaN
2,House,11,Steve.Bergquist@leg.wa.gov,Syd.Locke@leg.wa.gov,Syd Locke,Steve Bergquist,D,(360) 786-7862,2.0,JLOB 322,NaN,NaN
3,House,19,Brian.Blake@leg.wa.gov,Dawn.Thomas@leg.wa.gov,Dawn Thomas,Brian Blake,D,(360) 786-7870,2.0,LEG 437A,NaN,NaN
4,House,42,Vincent.Buys@leg.wa.gov,Amanda.Yager@leg.wa.gov,Amanda Yager,Vincent Buys,R,(360) 786-7854,2.0,JLOB 465,NaN,NaN
5,House,26,Michelle.Caldier@leg.wa.gov,Aaron.Hallenberg@leg.wa.gov,Aaron Hallenberg,Michelle Caldier,R,(360) 786-7802,2.0,LEG 122G,NaN,NaN
6,House,15,Bruce.Chandler@leg.wa.gov,Melinda.Bailey@leg.wa.gov,Melinda Bailey,Bruce Chandler,R,(360) 786-7960,1.0,LEG 427B,NaN,NaN
7,House,24,Mike.Chapman@leg.wa.gov,Sarah.Temples@leg.wa.gov,Sarah Temples,Mike Chapman,D,(360) 786-7916,1.0,LEG 132B,NaN,NaN
8,House,43,Frank.Chopp@leg.wa.gov,nora.burnes@leg.wa.gov,Nora Burnes,Frank Chopp,D,(360) 786-7920,2.0,LEG 339C,NaN,NaN
9,House,41,Judy.Clibborn@leg.wa.gov,Negheen.Kamkar@leg.wa.gov,Negheen Kamkar,Judy Clibborn,D,(360) 786-7926,2.0,JLOB 415,NaN,NaN


In [56]:
legislator_info_df = legislator_info.drop(['Unnamed: 10','Unnamed: 11'], axis=1)

In [57]:
legislator_info_df

,Chamber,District,Email,LA Email,Legislative Assistant,Member Name,Party,Phone,Position,Room
0,House,23,Sherry.Appleton@leg.wa.gov,Lisa.Hardy@leg.wa.gov,Lisa Hardy,Sherry Appleton,D,(360) 786-7934,1.0,LEG 132F
1,House,2,Andrew.Barkis@leg.wa.gov,Shelby.Pelon@leg.wa.gov,Shelby Pelon,Andrew Barkis,R,(360) 786-7824,1.0,LEG 122D
2,House,11,Steve.Bergquist@leg.wa.gov,Syd.Locke@leg.wa.gov,Syd Locke,Steve Bergquist,D,(360) 786-7862,2.0,JLOB 322
3,House,19,Brian.Blake@leg.wa.gov,Dawn.Thomas@leg.wa.gov,Dawn Thomas,Brian Blake,D,(360) 786-7870,2.0,LEG 437A
4,House,42,Vincent.Buys@leg.wa.gov,Amanda.Yager@leg.wa.gov,Amanda Yager,Vincent Buys,R,(360) 786-7854,2.0,JLOB 465
5,House,26,Michelle.Caldier@leg.wa.gov,Aaron.Hallenberg@leg.wa.gov,Aaron Hallenberg,Michelle Caldier,R,(360) 786-7802,2.0,LEG 122G
6,House,15,Bruce.Chandler@leg.wa.gov,Melinda.Bailey@leg.wa.gov,Melinda Bailey,Bruce Chandler,R,(360) 786-7960,1.0,LEG 427B
7,House,24,Mike.Chapman@leg.wa.gov,Sarah.Temples@leg.wa.gov,Sarah Temples,Mike Chapman,D,(360) 786-7916,1.0,LEG 132B
8,House,43,Frank.Chopp@leg.wa.gov,nora.burnes@leg.wa.gov,Nora Burnes,Frank Chopp,D,(360) 786-7920,2.0,LEG 339C
9,House,41,Judy.Clibborn@leg.wa.gov,Negheen.Kamkar@leg.wa.gov,Negheen Kamkar,Judy Clibborn,D,(360) 786-7926,2.0,JLOB 415


In [58]:
legislator_info_df.to_csv('Legislator_info.csv', sep = ',')

### Making a postgreSQL database

In [ ]:
#sticking it all in sql
#!psql -d postgres
#!create database WA_Leg;

In [1]:
dbname = 'WA_leg'
username = 'sundipta'

In [4]:
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://sundipta@localhost/WA_leg


In [5]:
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [81]:
leg_info_fromcsv = pd.read_csv("/Users/sundipta/Insight_notebooks/Legislator_info.csv")

In [82]:
leg_info_forsql = leg_info_fromcsv.rename(columns={'Unnamed: 0' : 'ID'})
leg_info_forsql.head()

,ID,Chamber,District,Email,LA Email,Legislative Assistant,Member Name,Party,Phone,Position,Room
0,0,House,23,Sherry.Appleton@leg.wa.gov,Lisa.Hardy@leg.wa.gov,Lisa Hardy,Sherry Appleton,D,(360) 786-7934,1.0,LEG 132F
1,1,House,2,Andrew.Barkis@leg.wa.gov,Shelby.Pelon@leg.wa.gov,Shelby Pelon,Andrew Barkis,R,(360) 786-7824,1.0,LEG 122D
2,2,House,11,Steve.Bergquist@leg.wa.gov,Syd.Locke@leg.wa.gov,Syd Locke,Steve Bergquist,D,(360) 786-7862,2.0,JLOB 322
3,3,House,19,Brian.Blake@leg.wa.gov,Dawn.Thomas@leg.wa.gov,Dawn Thomas,Brian Blake,D,(360) 786-7870,2.0,LEG 437A
4,4,House,42,Vincent.Buys@leg.wa.gov,Amanda.Yager@leg.wa.gov,Amanda Yager,Vincent Buys,R,(360) 786-7854,2.0,JLOB 465


In [83]:
leg_info_forsql.to_sql('legislator_info_table', engine, if_exists='replace')

In [93]:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM legislator_info_table;
"""
legislator_data_from_sql = pd.read_sql_query(sql_query,con)
legislator_data_from_sql.head()

,index,ID,Chamber,District,Email,LA Email,Legislative Assistant,Member Name,Party,Phone,Position,Room
0,0,0,House,23,Sherry.Appleton@leg.wa.gov,Lisa.Hardy@leg.wa.gov,Lisa Hardy,Sherry Appleton,D,(360) 786-7934,1.0,LEG 132F
1,1,1,House,2,Andrew.Barkis@leg.wa.gov,Shelby.Pelon@leg.wa.gov,Shelby Pelon,Andrew Barkis,R,(360) 786-7824,1.0,LEG 122D
2,2,2,House,11,Steve.Bergquist@leg.wa.gov,Syd.Locke@leg.wa.gov,Syd Locke,Steve Bergquist,D,(360) 786-7862,2.0,JLOB 322
3,3,3,House,19,Brian.Blake@leg.wa.gov,Dawn.Thomas@leg.wa.gov,Dawn Thomas,Brian Blake,D,(360) 786-7870,2.0,LEG 437A
4,4,4,House,42,Vincent.Buys@leg.wa.gov,Amanda.Yager@leg.wa.gov,Amanda Yager,Vincent Buys,R,(360) 786-7854,2.0,JLOB 465


In [57]:
bill_info_fromcsv = pd.read_csv("/Users/sundipta/Insight_notebooks/bill_descriptions.csv",index_col=0)

In [52]:
#bill_info_fromcsv = bill_info_fromcsv.drop(columns='ID')
#bill_info_fromcsv = bill_info_fromcsv.rename(columns = {'index':'ID'})

In [53]:
bill_info_fromcsv.head()

,ID,Active,Agency,Bill_ID,LongDescription,ShortDescription
0,0,True,House,1031,Concerning the use of unmanned aerial systems ...,UAVs near marine species
1,1,True,House,1032,Concerning the excise taxation of martial arts.,Martial arts excise taxation
2,2,False,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
3,3,True,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
4,4,False,House,1045,Addressing licensing and enforcement provision...,Uniform money services act


In [62]:
bill_info_fromcsv.to_sql('bill_info_table', engine, if_exists='replace')

In [63]:
#bill_info_fromcsv.to_csv('bill_descriptions.csv', sep = ',')

In [98]:
sql_query2 = """
SELECT * FROM bill_info_table;
"""
bill_data_from_sql = pd.read_sql_query(sql_query2,con)
bill_data_from_sql.head()

,index,ID,Active,Agency,Bill_ID,LongDescription,ShortDescription
0,0,0,True,House,1031,Concerning the use of unmanned aerial systems ...,UAVs near marine species
1,1,1,True,House,1032,Concerning the excise taxation of martial arts.,Martial arts excise taxation
2,2,2,False,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
3,3,3,True,House,1037,Concerning notice sent by and certain release ...,Tow truck notices & info.
4,4,4,False,House,1045,Addressing licensing and enforcement provision...,Uniform money services act


In [99]:
bill_votes_bien_fromcsv = pd.read_csv("/Users/sundipta/Insight_notebooks/current_bien_votes.csv")

In [100]:
bill_votes_forsql = bill_votes_bien_fromcsv.rename(columns={'Unnamed: 0' : 'ID'})
bill_votes_forsql.head()

,ID,Agency,Bill_ID,Legislator,Vote
0,0,House,1031,Appleton,1.0
1,1,House,1032,Appleton,1.0
2,2,House,1037,Appleton,1.0
3,4,House,1045,Appleton,1.0
4,6,House,1046,Appleton,1.0


In [101]:
bill_votes_forsql.to_sql('bill_votes_table', engine, if_exists='replace')

In [105]:
sql_query3 = """
SELECT * FROM bill_votes_table;
"""
bill_votes_from_sql = pd.read_sql_query(sql_query3,con)
bill_votes_from_sql.head()

,index,ID,Agency,Bill_ID,Legislator,Vote
0,0,0,House,1031,Appleton,1.0
1,1,1,House,1032,Appleton,1.0
2,2,2,House,1037,Appleton,1.0
3,3,4,House,1045,Appleton,1.0
4,4,6,House,1046,Appleton,1.0


In [24]:
bill_info = pd.read_csv("/Users/sundipta/Insight_notebooks/bill_descriptions.csv", index_col = 0)
bill_info['LongDescription'].iloc[1485:1490]

1485    NaN
1486    NaN
1487    NaN
1488    NaN
1489    NaN
Name: LongDescription, dtype: object

In [31]:
bill_info.iloc[[4,10,33]]['index']

4      4
10    11
33    36
Name: index, dtype: int64